In [ ]:
from groq import Groq
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
import os
import json


In [ ]:
load_dotenv(override=True)

In [ ]:
query = [{"role": "user", "content": "Give me a topic to generate a blog post on for my website named Visonalry Labs which explores the advancements in ai and machine learning."}]

In [ ]:
groq = Groq()
response = groq.chat.completions.create(
  model = "llama3-70b-8192",
  messages= query
)

answer = response.choices[0].message.content
display(Markdown(answer))

In [ ]:
responses = []
models = []

In [ ]:
topic = [{"role": "user", "content": answer}]

In [ ]:
response = groq.chat.completions.create(
  model = "deepseek-r1-distill-llama-70b",
  messages= topic
)

answer = response.choices[0].message.content
display(Markdown(answer))

responses.append(answer)
models.append(response.model)

In [ ]:
response = groq.chat.completions.create(
  model = "llama-3.1-8b-instant",
  messages = topic
)

answer = response.choices[0].message.content
display(Markdown(answer))

responses.append(answer)
models.append(response.model)

In [ ]:
response = groq.chat.completions.create(
  model = "llama-3.3-70b-versatile",
  messages = topic
)

answer = response.choices[0].message.content
display(Markdown(answer))

responses.append(answer)
models.append(response.model)


In [ ]:
gemini = OpenAI(api_key= os.getenv("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini.chat.completions.create(
    model="gemini-2.5-flash",
    messages= topic
    )

answer = response.choices[0].message.content
display(Markdown(answer))

responses.append(answer)
models.append(response.model)
print(models)


In [ ]:
together = ""
for index, (response, model) in enumerate(zip(responses, models)):
  together += f"Model {index+1}: {model}\nBlog Response: {response}\n\n"


In [ ]:
judge = f"""You are judging a competition between {len(models)} models.
Each model has been given this question:

{query}

Your job is to evaluate each response for clarity and strength of blog that has the most pin point information and engeaging factor for the reader, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [ ]:
judge_message = [{"role": "user", "content": judge}]

In [ ]:
model = "openai/gpt-4.1"

client = OpenAI(
    base_url="https://models.github.ai/inference",
    api_key=os.environ["GITHUB_TOKEN"],
)

response = client.chat.completions.create(
  model = model,
  messages=judge_message
)


In [ ]:

answer = response.choices[0].message.content
print(answer)
comp_result = json.loads(answer)
blog_results = comp_result['results']
print(blog_results)


In [ ]:

for index, result in enumerate(blog_results):
    model = models[int(result)-1]
    print(f"Rank {index+1}: {model}")

In [ ]:
winner = int(blog_results[0])
final_blog = f"Final Blog to publish is by: {models[winner-1]}\n {responses[winner-1]}"
display(Markdown(final_blog))